## This sample choose you own adventure (CYOA) application demonstrates using chains to accomplish the following:
1) Use the LangChain python library to implement chained queries with OpenAI Chatbot 
2) Use private context taken from the web - loading inventory from dndbeyond.com 
3) Implement Retrieval Augmented Generation (RAG) using in-memory vectorstore called FAISS
4) Use chains to implement history aware queries

In [11]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [12]:
load_dotenv()
llm = ChatOpenAI()

## Implement the retrieval chain and database
1) Scrape inventory from dndbeyond.com using WebBaseLoader and beautifulsoup4
2) Load invoentory into faiss vector store database
3) Use OpenAI embedding to measure relatedness of strings in vectorstore

In [47]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://www.dndbeyond.com/equipment?filter-search=Weapon')
inventory = loader.load()

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(inventory)

vectorstore = FAISS.from_documents(documents, embeddings)

## Load armory into embeddings
1) Create chain to pass Document to model

In [48]:
from langchain.chains.combine_documents import create_stuff_documents_chain

template = """Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(template)
document_chain = create_stuff_documents_chain(llm, prompt)

## Test that the document chain works
1) Pass a Document to document_chain
2) Model executes prompt using Document as context

In [49]:
from langchain_core.documents import Document

document_chain.invoke({
    "input":"List the names of the first 3 weapons",
    "context": [Document(page_content="List of weapons: 1) Axe 2) Slingshot, 3) Halbred 4) Sword 5) Javelin")]
})

'The names of the first 3 weapons are Axe, Slingshot, and Halberd.'

## Implement retrieval chain
1) Use vectorstore as Document source

In [50]:
from langchain.chains import create_retrieval_chain

vectorstore_retriever = vectorstore.as_retriever()
retrieval_chain = create_retrieval_chain(vectorstore_retriever, document_chain)

response = retrieval_chain.invoke({
    "input" : "List the names of all the available weapons in the following the format: 1. Weapon 1\n2. Weapon 2."
})

# We'll use this later
ai_message = response.get('answer')

## Test retrieval chain works
1) Use document store as source for model to execute input query
2) 'context' contains relevant Documents from vectorstore
3) 'answer' should contain result from llm to input question
4) Parse the output

In [51]:
response

# Uncomment to see parsed inventory
# response.get('answer').split('\n')

{'input': 'List the names of all the available weapons in the following the format: 1. Weapon 1\n2. Weapon 2.',
 'context': [Document(page_content='View Magic Items\n        \n\n\n\n\n\n\n\n\n\n              Equipment\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n                Search\r\n            \n\n\n\n\n\n\n\n\r\n                Cost\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\r\n                Weight\r\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\nFilter Equipment\n\n\n\n\n\n\nShow Advanced Filters\n\n\n\n\nReset All Filters\n\n\n\n\nName\nCost\nWeight\nAttributes\nNotes\n\n\n\n\n\n12Next\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n                            Antimatter Rifle\n                        \n\n\n\n\n                        Firearms Ranged Weapon\n                    \n\n\n\n\n\n\n\n                         -- \n                    \n\n\n\n\n\n\n\n                        10 lbs\n                    \n\n\n\n\n\n\n\n                        Necrotic\n  

## Conversation aware vectorstore retriever chain 
1) Maintain context for searching relevant embedding in vectorstore

In [52]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])

retriever_chain = create_history_aware_retriever(llm, vectorstore_retriever, prompt)


# Conversation aware document retriever chain
1) Maintain for document retriever query

In [61]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the users questions based on the below contenxt:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}")
])

document_chain = create_stuff_documents_chain(llm, prompt)

## Final product - here's the beautiful part
1) Create our final conversation aware vectorstore retriever chain
2) Chat history will be used in queries for both embedding AND the returned documents

In [65]:
conversation_retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

from langchain_core.messages import HumanMessage, AIMessage

chat_history = [
    HumanMessage(content="List the names of all the available weapons in the following the format: 1. Weapon 1\n2. Weapon 2."),
    AIMessage(content=ai_message)
]
response = conversation_retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "List the names of all the ranged weapons, and their weights, in following the format: 1. Weapon 1, (WEIGHT)\n2. Weapon 2 (WEIGHT)"
})

In [66]:
print(response.get('answer'))

inventory_raw = response.get('answer').split('\n')

# [item for item in inventory_raw if any (w in item for w in ['Melee Weapon', 'Ranged Weapon'])]

1. Antimatter Rifle (10 lbs)
2. Bad News (Exandria) (25 lbs)
3. Blowgun (1 lb)
4. Blunderbuss (Exandria) (10 lbs)
5. Boomerang (--)
6. Crossbow, hand (3 lbs)
7. Crossbow, heavy (18 lbs)
8. Crossbow, light (5 lbs)
9. Hand Mortar (Exandria) (--)
10. Hoopak (Ranged) (2 lbs)
11. Laser Pistol (2 lbs)
12. Laser Rifle (7 lbs)
13. Longbow (2 lbs)
14. Crossbow, light (--)
15. Dart (1/4 lb)
